In [4]:
import numpy as np
bases={'A':np.array([0,0,0,1]), 'C':np.array([0,0,1,0]), 'G':np.array([0,1,0,0]), 'T':np.array([1,0,0,0]), 'N':np.array([0,0,0,0]), 'Y':np.array([0,0,0,0]), 'R':np.array([0,0,0,0]), 'W':np.array([0,0,0,0]), 'S':np.array([0,0,0,0]), 'K':np.array([0,0,0,0])}

def one_hot_encode(seq, unique_elems,maxlen):
    seq_encoded=[]
    i=0
    for el in seq:
        # print(el)
        if i<maxlen:
            seq_encoded.append(unique_elems[el])
            i+=1
        
    return np.array(seq_encoded)

def one_hot_encode_2(y,num_classes):
    y_encoded=[]
    for el in y:
        y_encoded.append(np.eye(num_classes)[el])
    return np.array(y_encoded)



In [5]:
from Bio import SeqIO
unique_elems=dict()
x_data=[]
y_data=[]
minlen=100000
for sequence in SeqIO.parse('dataset.fasta', "fasta"):
    unique_elems[sequence.description.split()[1]+sequence.description.split()[2]]=0
    if len(sequence.seq)<minlen:
        minlen=len(sequence.seq)

# print(minlen)
print(unique_elems.keys())
i=0
for el in unique_elems.keys():
    unique_elems[el]=i
    i+=1
# print(unique_elems)
for sequence in SeqIO.parse('dataset.fasta', "fasta"):
    x_data.append(one_hot_encode(sequence.seq,bases,minlen))
    y_data.append(unique_elems[sequence.description.split()[1]+sequence.description.split()[2]])

x_data=np.array(x_data)
y_data=one_hot_encode_2(y_data,len(unique_elems))


print(x_data.shape)
    

dict_keys(['Acinetobacterbaumannii', 'Acinetobactercalcoaceticus', 'Bacteroidesfragilis', 'Candidaalbicans', 'Candidaauris', 'Candidaglabrata', 'Candidaparapsilosis', 'Cosenzaeamyxofaciens', 'Cryptococcusneoformans', 'Enterobactercloacae', 'Enterococcusfaecalis', 'Enterococcusfaecium', 'Escherichiacoli', 'Haemophilusinfluenzae', 'Klebsiellaaerogenes', 'Klebsiellaoxytoca', 'Klebsiellapneumoniae', 'Listeriamonocytogenes', 'Neisseriameningitidis', 'Pichiakudriavzevii', 'Thermoproteusthermophilus', 'Thermoproteustenax', 'Proteusalimentorum', 'Proteuscolumbae', 'Proteuscibarius', 'Proteusterrae', 'Motiliproteussediminis', 'Shimwelliapseudoproteus', 'Obesumbacteriumproteus', 'Proteusvulgaris', 'Proteusmirabilis', 'Proteushauseri', 'Proteuspenneri', 'Pseudomonasaeruginosa', 'Salmonellaenterica', 'Salmonellabongori', 'Serratiamarcescens', 'Staphylococcusaureus'])
(118, 1110, 4)


In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D,Reshape, LSTM
from tensorflow.keras.callbacks import EarlyStopping


In [8]:
model=Sequential()
model.add(LSTM(units=256, input_shape=(x_data.shape[1],x_data.shape[2])))
model.add(Reshape((-1,8)))
model.add(Conv1D(filters=128, kernel_size=12, activation='relu'))
model.add(Flatten())
model.add(Dense(units=len(unique_elems), activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='loss', patience=2, mode='min')
history = model.fit(x_data, y_data, epochs=1000, verbose=1, validation_data= None)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               267264    
_________________________________________________________________
reshape_1 (Reshape)          (None, 32, 8)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 21, 128)           12416     
_________________________________________________________________
flatten (Flatten)            (None, 2688)              0         
_________________________________________________________________
dense (Dense)                (None, 38)                102182    
Total params: 381,862
Trainable params: 381,862
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000


2022-01-02 15:51:28.510852: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-02 15:51:28.510997: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-02 15:51:28.865967: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-02 15:51:28.931439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-02 15:51:33.258929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 27s 4s/step - loss: 3.6273 - accuracy: 0.0508
Epoch 2/1000
4/4 [==============================] - 3s 607ms/step - loss: 3.5428 - accuracy: 0.1525
Epoch 3/1000
4/4 [==============================] - 2s 530ms/step - loss: 3.2901 - accuracy: 0.1356
Epoch 4/1000
4/4 [==============================] - 2s 468ms/step - loss: 3.2244 - accuracy: 0.1356
Epoch 5/1000
4/4 [==============================] - 2s 502ms/step - loss: 3.2261 - accuracy: 0.1356
Epoch 6/1000
4/4 [==============================] - 2s 466ms/step - loss: 3.2019 - accuracy: 0.1356
Epoch 7/1000
4/4 [==============================] - 2s 487ms/step - loss: 3.1953 - accuracy: 0.1356
Epoch 8/1000
4/4 [==============================] - 2s 612ms/step - loss: 3.1740 - accuracy: 0.1356
Epoch 9/1000
4/4 [==============================] - 2s 521ms/step - loss: 3.1695 - accuracy: 0.1441
Epoch 10/1000
4/4 [==============================] - 2s 505ms/step - loss: 3.1381 - accuracy: 0.1441
Epoch 11/1000


KeyboardInterrupt: 